Working Notebook for Final preprocessing script. The script will do the following:
1) Filter all AE zones and dissolve
2) Clean broken BFEs
3) Ensure all BFEs are single linestrings
4) Reset the BFEs after extension process
5) Split FLood Zone based on extended BFEs
6) Clip the FLood ZOne back to original extents

In [1]:
from new_methods import *

process_dict = {'CO': ['08013' , '08059', '08014', '08047', '08049', '08069', '08123', '08001', '08005', '08013', '08019', '08031', '08035', '08093', '08119'],
                'FL': ['12086', '12011', '12021', '12087', '12051', '12099'],
                'LA': ['22071', '22051', '22075', '22087', '22103', '22057', '22089', '22095', '22105']}

In [2]:
process_dict = {'CO': ['08059']}

for co, ls in process_dict.items():
    for l in ls:
        for fi in glob.glob(f'{co}_FZ\*{l}.shp'):
            selFZ = gpd.read_file(os.path.abspath(fi))
            print('Processing', fi)
            
        for fi in glob.glob(f'{co}_BFEs\*{l}.shp'):
            selBFE = gpd.read_file(os.path.abspath(fi))
            print('Processing', fi)
        # determine correct UTM projection from flood zone
        crs = utm_code(selFZ)

        # Locate all AE and A* zones with no Static values
        sel_ae = selFZ.loc[(selFZ['ZONE'].str.contains('A.')) & 
        (selFZ['ZONE'] != 'A99') & 
        (selFZ['ZONE'] != 'AO') & 
        (selFZ['ZONE'] != 'AR')
        ]

        sel_ae_dynamic = sel_ae.loc[sel_ae['BFE_STATIC'] == -9999.0]

        sel_ae_dis = sel_ae_dynamic.dissolve(by='FIPS')
        sel_ae_dis.to_crs(crs, inplace=True)
        sel_fsp = sel_ae_dis[['geometry']]
        sel_fsp.reset_index(inplace=True)

        # If there are no AE or A* Zones (i.e. dataframe is empty, continue on to the next county)
        if len(sel_fsp) > 0:
            # Reproject BFEs 
            selBFE.to_crs(crs, inplace=True)
            sel_bfe = selBFE[['ELEV', 'geometry']]
            sel_bfe = sel_bfe.drop_duplicates('geometry') # * delete late

            """ If main Index joins with other Indices, this means that a BFE is broken
             up into several linestrings. This cleaning process attempts to merge
            all linestrings into one per BFE """

            bfe_bfe = sel_bfe.sjoin(sel_bfe, how='left')
            bfe_bfe.reset_index(inplace=True)

            brok = bfe_bfe.loc[bfe_bfe['index'] != bfe_bfe['index_right']]
            sel_bfe.reset_index(inplace=True)

            if len(brok) > 0:
                print('Cleaning BFEs...')
                # df with both main geometry and adjacent geoms
                fix = brok.merge(sel_bfe, left_on='index_right', right_on='index')

                """Locating and gathering adjacent geometries to linestrings in the middle. These will account for 
                BFEs that have been broken up more than twice """
                # Series with indice count of spatial join
                vals = fix['index_x'].value_counts()

                # Obtaining the Indices that touch more than one linestring
                mid = fix.loc[fix['index_x'].isin(vals.index[vals.gt(1)])]

                # Groupby Main Index and collect adj. geoms into list
                mid_gb = mid.groupby('index_x')['geometry_y'].apply(list)

                # Groupby Main Index and collect adj. indices into list
                mid_adj = mid.groupby('index_x')['index_right'].apply(list)

                # Build df from groupby series to join and obtain key geometry
                mid_df = pd.DataFrame(mid_gb)
                mid_df.reset_index(inplace=True)
                mid_join = mid_df.merge(sel_bfe, left_on='index_x', right_index=True)

                """ Adjacent IDS to Middle that need to be removed since these will be apart of the 3 linestring merge """
                # Creating list of Main Indices that touch more than one linestring
                mid_ids = mid['index_x'].drop_duplicates().to_list()

                # Creating list of adjacent indices that touch Main Index
                adj_list = [ids for m in mid_adj.to_list() for ids in m]
                adj_list = adj_list + mid_ids  

                """ These should be 2 linestrings ONLY """
                # Remove ids from adj_list, leaving ids that touch only One other linestring
                fix_rem = fix.loc[~fix['index_x'].isin(adj_list)]

                """ Correct 2 linestring breaks """
                fix_rem['new_geom'] = fix_rem.apply(lambda x: linemerge(list((x.geometry_x, x.geometry_y))), axis=1 )
                fix_df = fix_rem[['index_x', 'ELEV_left', 'new_geom']].rename(columns={'index_x': 'index', 'new_geom': 'geometry', 'ELEV_left': 'ELEV'}).set_geometry('geometry', crs=crs)

                # This removes the other pair in the 1-to-1 join
                fix_df = fix_df.drop_duplicates('geometry')

                """ Correct 3 (or more) linestring breaks """
                if len(mid_join) > 0:
                    mid_join['new_geom'] = mid_join.apply(lambda x: linemerge(x['geometry_y'] + [x['geometry']]), axis=1)
                    mid_join = mid_join[['index_x', 'ELEV', 'new_geom']].rename(columns={'index_x': 'index', 'new_geom': 'geometry'}).set_geometry('geometry', crs=crs)

                    bfe_cleaned = pd.concat([mid_join, fix_df], names=['ELEV', 'geometry'])
                    bfe_c_geom = bfe_cleaned[['geometry']]
                   
                    # spatial join cleaned geometry to original bfe df
                    clean_join = sel_bfe.sjoin(bfe_c_geom, how='left')
                    
                    # Locate bfes that were not cleaned
                    sel_bfe = clean_join.loc[clean_join['index_right'].isnull(), ['ELEV', 'geometry']]

                    # Final Cleaned BFE dataframe
                    bfe = pd.concat([bfe_cleaned, sel_bfe], ignore_index=True)
                else:
                    fix_df = fix_df[['ELEV', 'geometry']]
                    bfe_c_geom = fix_df[['geometry']]
                    clean_join = sel_bfe.sjoin(bfe_c_geom, how='left')
                    sel_bfe = clean_join.loc[clean_join['index_right'].isnull(), ['ELEV', 'geometry']]
                
                    bfe = pd.concat([fix_df, sel_bfe], ignore_index=True)
                
                
            else:
                bfe = sel_bfe[['ELEV', 'geometry']]

            # final removal of any multiline artifacts
            bfe = remove_multiline_BFE(bfe)
            
            # If conversion occured, there would by multiple linestrings that now have duplicate geometries. Need removal
            bfe = bfe.drop_duplicates('geometry')

            # repair geometries before following phase
            bfe['rgeom'] = bfe.apply(lambda x: make_valid(x.geometry), axis=1)
            bfe = bfe[['ELEV', 'rgeom']].rename(columns={'rgeom': 'geometry'}).set_geometry('geometry', crs=crs)
                


            # Extend BFEs over FSP Poly then reset bfe to new position
            print('Extending BFE...')
            bfe_ext, short_bfe, corrupt = bfe_extend(bfe, sel_fsp, crs=crs)

            if len(corrupt) > 0:
                print(l, 'Corrupt BFE')
                print(corrupt)


            print('Splitting FSP...')
            # Split FSP Poly by extended BFEs
            fsp_s = split_fsp(sel_fsp, bfe_ext, crs=crs)

            print('Cleaning FSP...')
            # Remove slivers due to overlaps from extended BFEs
            fsp_s = fsp_s.overlay(sel_fsp)
            fsp_s = fsp_s.loc[fsp_s.geom_type == 'Polygon']
            fsp_s = fsp_s.loc[fsp_s.area > 1] # Removing small artifacts from cleaning and split process
            fsp_s = fsp_s[['geometry']]
            fsp_s.reset_index(inplace=True)

            fsp_s = fsp_s.to_crs(4326)
            fsp_s.to_file(f'{l}_FloodZone_split_3ext_10_90.shp')

            # add short bfes to final bfe dataframe
            bfe = pd.concat([bfe_ext, short_bfe])
            bfe = bfe.to_crs(4326)
            bfe.to_file(f'BFE_{l}_3ext_10_90.shp')

        else:
            continue
        
                    


                    

Processing CO_FZ\FM_08059.shp
Processing CO_BFEs\EL_08059.shp
Cleaning BFEs...
Extending BFE...
polygon to line 0:00:00.006830
ends 0:00:01.420926
parse end points 0:00:04.572774
create start point for extapolation 0:00:05.707008
create extrapolation 0:00:07.656292
intersection 0:00:41.643113
remove empty linestrings 0:00:00.019538
get nearest point 0:00:10.360350
create ext_line 0:00:11.608875
combine extension 0:00:08.166999
new ends 0:00:01.401370
parse new ends 0:00:04.670918
buffer 0:00:05.490187
symmetric difference 0:02:14.062722
create rep point 0:00:06.677684
final ext line 0:00:08.225014
create final line 0:00:05.982375
final clean up 0:00:00.124994
Splitting FSP...
Cleaning FSP...


In [3]:
short_bfe.to_file('short.shp')